## 初始化环境, 固定随机种子

In [ ]:
import os
import pathlib

import torch
import torchvision as tv

import utils
from transformer_flow import Model


utils.set_random_seed(100)
notebook_output_path = pathlib.Path('runs/notebook')
"""输出路径"""

## 设置超参数

In [ ]:
# region 数据集元信息
dataset = 'mnist'
"""数据集"""
num_classes = 10
"""种类数"""
img_size = 28
"""图像大小"""
channel_size = 1
"""图像通道数"""
# endregion

# region 训练超参数
# we use a small model for fast demonstration, increase the model size for better results
patch_size = 4
"""图片块大小"""
channels = 128
"""隐藏层通道数"""
blocks = 4
"""MetaBlock数目"""
layers_per_block = 4
"""每个MetaBlock的层数"""
# try different noise levels to see its effect
noise_std = 0.1
"""噪声标准差"""

batch_size = 256
"""批量大小"""
lr = 2e-3
"""学习率"""
# increase epochs for better results
epochs = 100
"""训练步长"""
sample_freq = 10
"""采样频率"""
# endregion

# region 设备选择
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'  # if on mac
else:
    device = 'cpu'  # if mps not available
print(f'using device {device}')
# endregion

# 固定的噪声和固定目标标签, 用于检验生成效果
fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
"""固定噪声, 用于检验生成效果.

    shape: (Batch, C*P*P), 其中 Batch = num_classes * 10, L = W*W/P/P
"""
fixed_y = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, 10).flatten()


transform = tv.transforms.Compose([
    tv.transforms.Resize((img_size, img_size)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5,), (0.5,))
])
data = tv.datasets.MNIST('.', transform=transform, train=True, download=True)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)

model = Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size,
              channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
              num_classes=num_classes).to(device)

optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.95), lr=lr, weight_decay=1e-4)
lr_schedule = utils.CosineLRSchedule(optimizer, len(data_loader), epochs * len(data_loader), 1e-6, lr)

model_name = f'{patch_size}_{channels}_{blocks}_{layers_per_block}_{noise_std:.2f}'
sample_dir = notebook_output_path / f'{dataset}_samples_{model_name}'
ckpt_file = notebook_output_path / f'{dataset}_model_{model_name}.pth'
sample_dir.mkdir(exist_ok=True, parents=True)

for epoch in range(epochs):
    losses = 0
    for x, y in data_loader:
        # x.shape = (B, C, H, W), y.shape = (B)
        x = x.to(device)
        # 加噪声
        eps = noise_std * torch.randn_like(x)
        x = x + eps
        y = y.to(device)

        # 重置梯度
        optimizer.zero_grad()

        # 前向传播
        z, outputs, logdets = model(x, y)

        # 计算损失
        loss = model.get_loss(z, logdets)

        # 计算梯度
        loss.backward()

        # 参数优化
        optimizer.step()

        # 更新学习率
        lr_schedule.step()

        # 统计损失
        losses += loss.item()

    print(f"epoch {epoch} lr {optimizer.param_groups[0]['lr']:.6f} loss {losses / len(data_loader):.4f}")
    print('layer norms', ' '.join([f'{z.pow(2).mean():.4f}' for z in outputs]))
    if (epoch + 1) % sample_freq == 0:
        with torch.no_grad():
            samples = model.reverse(fixed_noise, fixed_y)
        tv.utils.save_image(samples, sample_dir / f'samples_{epoch:03d}.png', normalize=True, nrow=10)
        tv.utils.save_image(model.unpatchify(z[:100]), sample_dir / f'latent_{epoch:03d}.png', normalize=True, nrow=10)
        print('sampling complete')
torch.save(model.state_dict(), ckpt_file)

using device cuda


In [ ]:
# now we can also evaluate the model by turning it into a classifier with Bayes rule, p(y|x) = p(y)p(x|y)/p(x)
data = tv.datasets.MNIST('.', transform=transform, train=False, download=False)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=False)
num_correct = 0
num_examples = 0
for x, y in data_loader:
    x: torch.Tensor = x.to(device)
    y: torch.Tensor = y.to(device)
    eps = noise_std * torch.randn_like(x)
    x = x.repeat(num_classes, 1, 1, 1)
    y_ = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, y.size(0)).flatten()
    with torch.no_grad():
        z, outputs, logdets = model(x, y_)
        losses = 0.5 * z.pow(2).mean(dim=[1, 2]) - logdets  # keep the batch dimension
        pred = losses.reshape(num_classes, y.size(0)).argmin(dim=0)
    num_correct += (pred == y).sum()
    num_examples += y.size(0)
print(f'Accuracy %{100 * num_correct / num_examples:.2f}')